In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [7]:
!pip install -qU "langchain[groq]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 4.4 MB/s eta 0:00:00


In [8]:
!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 3.7 MB/s eta 0:00:00a 0:00:01


In [9]:
from langchain_community.graphs import Neo4jGraph, NetworkxEntityGraph
from langchain.chains import GraphQAChain
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate


In [10]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GROQ_API_KEY")
secret_value_1 = user_secrets.get_secret("LANGSMITH_KEY")
NEO4J_PASSWORD = user_secrets.get_secret("NEO4J_PASSWORD")
NEO4J_URI = user_secrets.get_secret("NEO4J_URI")
NEO4J_USERNAME = user_secrets.get_secret("NEO4J_USERNAME")



In [11]:
import os
os.environ["GROQ_API_KEY"] = secret_value_0
os.environ["LANGCHAIN_API_KEY"] = secret_value_1
os.environ["LANGCHAIN_PROJECT"] = "Game Bot Detection Framework"
os.environ["LANGCHAIN_TRACING_V2"] = "true" 

In [8]:
def create_networkx_graph_from_neo4j(neo4j_graph):
    # Create an empty directed graph
    G = nx.DiGraph()
    
    # Query to get nodes and relationships from Neo4j
    nodes_query = "MATCH (n) RETURN id(n) AS id, labels(n) AS labels"
    relationships_query = "MATCH (a)-[r]->(b) RETURN id(a) AS source, id(b) AS target, type(r) AS relation"

    # Fetch nodes
    nodes = neo4j_graph.query(nodes_query)
    for node in nodes:
        G.add_node(node['id'], labels=node['labels'])

    # Fetch relationships
    relationships = neo4j_graph.query(relationships_query)
    for rel in relationships:
        G.add_edge(rel['source'], rel['target'], relation=rel['relation'])

    return NetworkxEntityGraph(G)


In [12]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.3-70b-versatile")

In [13]:
graph = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD
)
# Create NetworkX graph from Neo4j
#networkx_graph = create_networkx_graph_from_neo4j(graph)

<ipython-input-13-a30fcd6467a8>:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


In [14]:
def extract_player_features(player_id: str) -> dict:
    """
    Extracts features from the knowledge graph for a given player.
    """
    query = f"""
    MATCH (p:Player {{Actor: toInteger('{player_id}')}})
    RETURN
        p.Actor AS player_id,
        p.A_Acc AS a_acc,
        p.Login_day_count AS login_day_count,
        p.Logout_day_count AS logout_day_count,
        p.Playtime AS playtime,
        p.playtime_per_day AS playtime_per_day,
        p.avg_money AS avg_money,
        p.Login_count AS login_count,
        p.ip_count AS ip_count,
        p.Max_level AS max_level
    """
    results = graph.query(query)
    if results:
        return results[0]
    else:
        return {}

In [15]:
prompt = ChatPromptTemplate.from_template(
"""
<|system|>
You are an expert game analyst tasked with identifying bots in an online game by analyzing player statistics. Your role is to generate an anomaly score (0-100, higher score means more likely to be a bot) and provide concise reasoning based on the player data.

Here is how to interpret each player statistic. Provide context for how each statistic informs your score. Focus on how these characteristics point to a bot.

- **Actor**: The player's unique identifier.
- **A_Acc**: Account ID of the player; may provide clues about account creation patterns.
- **Login_day_count**: Number of distinct days the player has logged in. Focus on if low compared to other statitics, this will indicate bot behaviour.
- **Logout_day_count**: Number of days the player logged out. Should closely align with login days; inconsistencies could be a sign.
- **Playtime**: Total playtime in seconds. An unusually high total compared to login days could signal a bot farming.
- **playtime_per_day**: Average playtime per day in seconds. High values here are key identifiers of 24/7 bots.
- **avg_money**: Average amount of in-game currency. Focus on this, as extreme values here could mean exploits or excessive farming.
- **Login_count**: Total number of logins; high activity would indicate a bot constantly repeating logins.
- **ip_count**: Number of unique IP addresses used. A very low counts or a high could signal an anomaly
- **Max_level**: Maximum level achieved by the player. This, with the above information would provide an indicator on its anomaly.

Consider:

1.  Is the "Login_day_count" very low, which would indicate little human input.
2.  Examine Playtime, playtime_per_day, and avg_money to uncover indications of farming or exploiting.
3. Look at the ip_count. An extremely large number or low number would indicate bot behaviour

Based on those descriptions, you will be given the following information and need to determine these two things

1.  Determine the **anomaly score** (0-100): A higher score indicates a greater likelihood of the player being a bot. This MUST be provided

2.  Provide a **detailed explanation** of your reasoning. This MUST be provided

Your final output **MUST** follow this structure:

Anomaly Score: [your anomaly score 0-100]
Reasoning: [Your reasoning based on patterns from the statistics. Be as clear as possible]

Respond concisely and directly.

<|user|>
Here is the player data:

Actor: {actor}
A_Acc: {a_acc}
Login_day_count: {login_day_count}
Logout_day_count: {logout_day_count}
Playtime: {playtime}
playtime_per_day: {playtime_per_day}
avg_money: {avg_money}
Login_count: {login_count}
ip_count: {ip_count}
Max_level: {max_level}

Anomaly Score:
Reasoning:
""")

In [16]:
def assess_bot_likelihood(player_data) -> tuple[int, str, str]:
    """
    Assesses the likelihood of a player being a bot using LLM and extracts score and reasoning.
    Returns a tuple of (anomaly_score, reasoning, full_analysis).
    """
    # Format the prompt
    formatted_prompt = prompt.format_messages(
        actor=player_data['player_id'],
        a_acc=player_data['a_acc'],
        login_day_count=player_data['login_day_count'],
        logout_day_count=player_data['logout_day_count'],
        playtime=player_data['playtime'],
        playtime_per_day=player_data['playtime_per_day'],
        avg_money=player_data['avg_money'],
        login_count=player_data['login_count'],
        ip_count=player_data['ip_count'],
        max_level=player_data['max_level']
    )

    # Call the LLM directly
    response = llm.invoke(formatted_prompt)
    full_analysis = response.content

    # Extract Anomaly Score and Reasoning
    try:
        score_line = next(line for line in full_analysis.split('\n') if "Anomaly Score" in line)
        anomaly_score = int(score_line.split(":")[1].strip())

        reasoning = "\n".join(full_analysis.split('\n')[1:])  # All lines after the score
    except Exception as e:
        anomaly_score = None
        reasoning = f"Could not reliably parse LLM response: {str(e)}"

    return anomaly_score, reasoning, full_analysis

In [17]:
def generate_bot_report(player_ids: list[str]) -> list[dict]:
    """
    Generates a report for a list of player IDs, including anomaly scores and reasoning.
    """
    report = []
    for player_id in player_ids:
        player_data = extract_player_features(player_id)
        if player_data:
            anomaly_score, reasoning, full_analysis = assess_bot_likelihood(player_data)

            report.append({
                "player_id": player_id,
                "anomaly_score": anomaly_score,
                "reasoning": reasoning,
                #"full_analysis": /full_analysis  # Optional - adds full LLM output
            })
        else:
            report.append({
                "player_id": player_id,
                "anomaly_score": None,
                "reasoning": "No data found for this player.",
                "full_analysis": None
            })
    return report


In [19]:
player_ids_to_check = ['428616', '433137', '447250', '416536', '441912']  # Replace with actual player IDs
bot_report = generate_bot_report(player_ids_to_check)

for record in bot_report:
    print(f"Player ID: {record['player_id']}")
    print(f"Anomaly Score: {record['anomaly_score']}")
    print(f"Reasoning: {record['reasoning']}")
    print("-" * 40)

Player ID: 428616
Anomaly Score: 40
Reasoning: Reasoning: The player's "Login_day_count" is relatively low at 4 days, which could indicate limited human input, but it's not extremely low. The "playtime_per_day" is high at 2179 seconds (approximately 36 minutes), which might suggest automated play, but it's not excessively high. The "avg_money" is relatively low, which doesn't strongly indicate farming or exploiting. The "ip_count" of 2 is moderate and doesn't raise significant concerns. The "Login_count" and "Logout_day_count" are close, which is a normal pattern. The "Max_level" of 10 is moderate, considering the playtime. Overall, while there are some potential indicators of bot-like behavior, they are not strong enough to conclusively identify the player as a bot, hence a moderate anomaly score of 40.
----------------------------------------
Player ID: 433137
Anomaly Score: 70
Reasoning: Reasoning: The player's data suggests a high likelihood of bot behavior due to several indicator

In [20]:
player_ids_to_check = ['427135', '431622', '213412', '419781', '345701']
bot_report = generate_bot_report(player_ids_to_check)

for record in bot_report:
    print(f"Player ID: {record['player_id']}")
    print(f"Anomaly Score: {record['anomaly_score']}")
    print(f"Reasoning: {record['reasoning']}")
    print("-" * 40)

Player ID: 427135
Anomaly Score: 80
Reasoning: Reasoning: The player's data suggests a high likelihood of being a bot due to several indicators. Firstly, the playtime_per_day is exceptionally high (73059.8571 seconds, approximately 20.3 hours), which is unrealistic for a human player and indicates 24/7 activity, a common trait of bots. Additionally, the total playtime (511419 seconds, approximately 142 hours) in just 7 distinct login days is unusually high, further supporting the notion of excessive farming or bot-like behavior. The avg_money is also relatively high, which could be a sign of exploiting or farming in-game currency. The low ip_count (2) could indicate that the player is using a static IP or a VPN, which is sometimes used by bots to avoid detection. The consistency between Login_day_count and Logout_day_count, as well as the moderate Login_count, do not strongly indicate bot behavior on their own but do not contradict the other evidence either. The Max_level achieved (43)

In [21]:
player_ids_to_check = ['385163', '454690', '233533', '263292', '202735']
bot_report = generate_bot_report(player_ids_to_check)

for record in bot_report:
    print(f"Player ID: {record['player_id']}")
    print(f"Anomaly Score: {record['anomaly_score']}")
    print(f"Reasoning: {record['reasoning']}")
    print("-" * 40)

Player ID: 385163
Anomaly Score: 40
Reasoning: Reasoning: The player's data presents some indicators that could suggest bot-like behavior, but they are not conclusive. The "Login_day_count" and "Logout_day_count" are consistent, which is a positive sign of human behavior. The "playtime_per_day" is relatively high at 5472 seconds (approximately 1.5 hours), which could indicate a dedicated player but is not excessively high to immediately suggest a bot. The "avg_money" is moderate, and the "Max_level" achieved is 27, which, without knowing the game's level cap, does not immediately indicate botting. The "ip_count" matches the "Login_day_count," which could suggest the player logs in from the same location each day, a behavior consistent with human players. However, the "Login_count" of 46 over 16 distinct days could indicate some level of automation or intense dedication, contributing to the anomaly score. Overall, while there are some suspicious patterns, they are not strong enough to c